# Building an Assimilation System with pyPDAF without parallelisation

---



pyPDAF is a Python interface to PDAF (Parallel Data Assimilation Framework). The framework is mainly designed for ensemble data assimilation systems with high-dimensional complex weather and climae models. It has applications to both research and operational purposes. The Python interface allows for the use of PDAF in Python, a flexible and rich environment.

As per its name, the framework is designed with the aim to implement efficient parallelised data assimilation system. In this practical, we provide a step-by-step tutorial on constructing a DA system with pyPDAF without using parallelisation. This tutorial is a simplification of the code used in `example/online` directory.

## Install pyPDAF
---
Before discussing the DA system construction, let us install pyPDAF first. If you are familiar with Python, you might have the package manager `conda` installed. This can be obtained using `anaconda` or `miniconda`.

### On local computer:

In your terminal or anaconda prompt, run `conda create -n pypdaf -c conda-forge yumengch::pypdaf conda-forge::jupyter`.

You can then open this notebook using the command `jupyter notebook`


### On Google Colab (skip this section when you're not using Google Colab):

The following step will install `conda` on the Google Colab. Here, as the conda installation on Google Colab is a bit awkward. We build pyPDAF from source code.

In [ ]:
!git clone --recurse-submodules https://github.com/yumengch/pyPDAF.git # get source code
# change to pyPDAF directory
%cd pyPDAF
# build pyPDAF. This could take a few minutes
!CC=mpicc FC=mpifort python -m pip install . --config-settings=setup-args="-Dblas_lib=['openblas',]"
# return original directory
%cd ..

To provide a better view of PDAF output, we have to use wurlitzer

## Model and Observations

In a typical data assimilation application, one usually has a model that can simulate the system of interest. For example, in the numerical weather prediction, an atmosphere model is available. The same applies to ocean, sea ice, biogeochemistry and land surface models. These models usually provide a priori estimate (forecast) of the state of the system in sequential data assimilation schemes, or an initial guess/background state in variational methods. The motivation of data assimilation system is to obtain a good estimate of the state of the system with observations, which makes up another important component of data assimilation.

When constructing a data assimilation system in pyPDAF, providing the model state and observation as well as their connection and uncertainty information is the key task. Before we get into pyPDAF, let's look at the example model and observations in this tutorial.

### Model

In this tutorial, a 2D model that propagates a sine wave along  in a rectangular domain is used for demonstration purpose. The model integration can be represented by a `step` function:

In [ ]:
"""define the model integration step"""
import numpy as np

def step(field: np.ndarray) -> np.ndarray:
    """Roll array elements of i-th time step along a the first axis."""
    return np.roll(field, shift=1, axis=-2)

In the tutorial, the model has
- Spatial Domain: Two dimensional domain grid domain with $(nx \times ny) = (36 \times 18)$ grid points
- Total steps: we will run this model by `nsteps = 18` time steps

In [ ]:
import numpy as np
# define the array for model field
nsteps = 18 # total time steps
nx = 36 # 36 columns
ny = 18 # 18 rows
# initial condition + 18 time steps, 18 rows and 36 columns
field = np.zeros((nsteps + 1, ny, nx))

For the sake of simplicity, a true initial condition is provided:

In [ ]:
""" this is a way to get all required data without using git and introducing additional libraries"""
import os
import urllib.request
# create input data directory
os.makedirs('inputs_online', exist_ok=True)
link_to_files = 'https://raw.githubusercontent.com/yumengch/pyPDAF/refs/heads/main/example/inputs_online/'
# get the initial truth
urllib.request.urlretrieve(f'{link_to_files}/true_initial.txt', os.path.join('inputs_online','true_initial.txt'))

We can read the initial condition to model field array:

In [ ]:
"""read the initial condition of the model field"""
field[0] = np.loadtxt(os.path.join('inputs_online','true_initial.txt'))

Now, we can visualise the model evolution:

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
fig = plt.figure('animation')
ax = fig.add_subplot(111)
pc = ax.pcolormesh(field[0], cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(pc, ax=ax)

def draw_model(i):
  """Draw each model step
  """
  # run the model
  field[i+1] = step(field[i])
  pc.set_array(field[i+1])
  ax.set_title(f'Model step {i+1}')
  return pc,

# make an animation
anim = animation.FuncAnimation(fig, draw_model, frames=nsteps, interval=1000, blit=True)
plt.close(fig)
HTML(anim.to_html5_video())

### Observations

Observations are given for each time steps stored in 'obs_step*.txt'

In [ ]:
"""retrieve observations"""
for i in range(nsteps):
    urllib.request.urlretrieve(f'{link_to_files}/obs_step{i+1}.txt', os.path.join('inputs_online',f'obs_step{i+1}.txt'))

- Among 648 grid points only 28 grid points have observations. This is because most DA methods can estimate grid points without observations by assuming each grid points form a multi-variate Gaussian distribution.
- Compared to the truth, these observations have an error of 0.5
- The retrieved data use `-999` as missing values

In [ ]:
# define observation array
obs = np.ma.zeros((nsteps, ny, nx))
fig = plt.figure('animationObs')
ax = fig.add_subplot(111)
pc = ax.pcolormesh(obs[0], cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(pc, ax=ax)
def draw_model(i):
  """Draw obs. at each model step
  """
  obs[i] = np.loadtxt(os.path.join('inputs_online', f'obs_step{i+1}.txt'))
  obs[i] = np.ma.masked_where(np.isclose(obs[i], -999.), obs[i])
  pc.set_array(obs[i])
  ax.set_title(f'Observation at step {i+1}')
  return pc,

# make an animation
anim = animation.FuncAnimation(fig, draw_model, frames=18, interval=1000, blit=True)
plt.close(fig)
HTML(anim.to_html5_video())

In this tutorial, we can actually obtain the truth,  

In [ ]:
for i in range(18):
    # get all truth
    urllib.request.urlretrieve(f'{link_to_files}/true_step{i+1}.txt', os.path.join('inputs_online', f'true_step{i+1}.txt'))

In comparison to the truth, we can check the observation error and calculate it as a sanity check.

In [ ]:
# calculate the root mean squared obs. err in time
err = np.sqrt(np.sum((field[1:] - obs)**2, axis=0)/nsteps)
# plot the observation error
fig = plt.figure('R')
ax = fig.add_subplot(111)
pc = ax.pcolormesh(err, cmap='Blues', vmin=0., vmax=1.)
ax.set_title(f'Spatial averaged obs. err is {np.round(np.mean(err), 3)}')
fig.colorbar(pc, ax=ax)

## Set up a data assimilation system using pyPDAF

Now, we can construct our data assimilation systems. As in many other Python programs, we use third-party packages for the system. This is one of the benefits of a Python system as many functionalities are available. In this tutorial, `pyPDAF`, `numpy` is used. As we do not utilise the parallel features of pyPDAF, `mpi4py` is not required.

In [ ]:
%%capture
"""For the sake of compatibility of running it in Google Colab, we install wurlitzer here.
   This is not essential if you run it on your local computer, but you will need to remove
   all code related to wurlitzer in this notebook."""
# wurlitzer is a package that allows us to see PDAF output
!pip install wurlitzer

In [ ]:
import pyPDAF
import numpy as np
from wurlitzer import pipes

### Initialise PDAF

The initialisation of PDAF tells PDAF the our choice of data assimilation algorithms, ensemble size, inflation factor, and the dimension of the state vector. Currently (up to PDAF V2.3.1), any DA methods in PDAF require these options. However, some methods can only be executed with additional parameters. This is done by [`PDAF.init`](https://yumengch.github.io/pyPDAF/PDAF.html#pyPDAF.PDAF.init).

In this tutorial, the **error space transform Kalman filter (ESTKF)** is used with **9** ensemble members. We will estimate the state of every model grid point, which gives us a state vector with the size of nx × ny = 36 × 18 = 648. The `filtertype` and `subtype` here specifies the DA method and will be given as an argument in `pyPDAF.PDAF.init` function. A full list of supported methods can be found in [PDAF wiki](https://pdaf.awi.de/trac/wiki/AvailableOptionsforInitPDAF). We first put these information into Python variables.

In [ ]:
# using error space transform Kalman filter (ESTKF)
filtertype = 6
# standard form
subtype = 0
# dimension of the state vector
# if model is parallelised, this is the dimension of state vector on each process
dim_state_p = nx*ny
# number of ensemble members
dim_ens = 9
# forget factor
forget_factor = 1.0

In addition to the above information, [`pyPDAF.init`](https://yumengch.github.io/pyPDAF/_autosummary/pyPDAF.init.html#pyPDAF.init) also asks for an **initial ensemble** for PDAF. This information is given by the user-supplied function [py__init_ens_pdaf](https://yumengch.github.io/pyPDAF/user_desc/py__init_ens_pdaf.html). These functions have fixed interface. Therefore
- the input arguments and return variables should not be changed.
- only the value of returned variables should be changed

Documentation of the input arguments and return variable of this function can be found in [pyPDAF documentation](https://yumengch.github.io/pyPDAF/_autosummary/pyPDAF.init.html#pyPDAF.init). In this user-supplied function, the primary purpose is to fill the `ens_p` array. The `ens_p` is an array allocated by PDAF to perform data assimilation. Pure input arguments such as `filtertype`, `dim_p`, and `dim_ens` are given by PDAF. These information is given to PDAF when we call `pyPDAF.init`. When we implement the user-supplied function, they can be difficult to access. Therefore, PDAF provide these variables as input argument in the user-supplied function. For example, in this tutorial, the initial ensemble is read from text files. The `dim_ens` argument helps us the set up a loop to read these text files.

In real applications, we may encouter different scenarios. For example, one may need to use algorithms to generate perturbations to create an initial ensemble from a model trajectory, see [PDAF functionality](https://pdaf.awi.de/trac/wiki/EnsembleGeneration). If one have an ensemble of model restart file, we can also simply return the variables in this user-supplied function.

In [ ]:
def init_ens_pdaf(filtertype, dim_p, dim_ens, state_p, uinv, ens_p, status_pdaf):
    """Here, only ens_p variable matters while dim_p and dim_ens defines the
    size of the variables. uinv, state_p are not used in this example.

    status_pdaf is used to handle errors which we will not do it in this example.
    """
    # get initial ensemble
    for i in range(dim_ens):
        urllib.request.urlretrieve(f'{link_to_files}/ens_{i+1}.txt',
                                   os.path.join('inputs_online', f'ens_{i+1}.txt'))
        ens_p[:, i] = np.loadtxt(os.path.join('inputs_online', f'ens_{i+1}.txt')).ravel()
    return state_p, uinv, ens_p, status_pdaf

With these information, we can call PDAF function `pyPDAF.init` to initialise the DA system.

In [ ]:
# this gives the verbose level of the PDAF, here we use 3 which is very verbose
screen = 3
# current step of the model which is 0
current_step = 0

with pipes() as (out, err):
    _, _, status = pyPDAF.init(filtertype, subtype, current_step,
                            np.array([dim_state_p, dim_ens], dtype=np.intc), 2,
                            np.array([forget_factor, ]), 1,
                            py__init_ens_pdaf=init_ens_pdaf,
                            in_screen=screen)
# print PDAF screen output
print (out.read())

### Distribution of the ensemble from PDAF
After PDAF initialisation, PDAF should distribute the ensemble initialised in `init_ens_pdaf` to the model to initialise the following forecast. This is accomplished by [`pyPDAF.init_forecast`](https://yumengch.github.io/pyPDAF/_autosummary/pyPDAF.init_forecast.html#pyPDAF.init_forecast) function.

This function depends on three user-supplied functions that are executed in the following sequence:
1. Processing the initial PDAF ensemble by user-supplied function [`py__prepoststep_state_pdaf`](https://yumengch.github.io/pyPDAF/user_desc/py__prepoststep_pdaf.html). This is because the ensemble stored in PDAF may not satisify some physical constraints such as balance conditions (hydrostatic balance/constitutive relations) or boundness (percentage values/chemical concentrations).
2. PDAF should distribute the initial ensemble to the model for following model forecasts in [`py__distribute_state_pdaf`](https://yumengch.github.io/pyPDAF/user_desc/py__distribute_state_pdaf.html). This function relates the model field to the state vector used by PDAF.
3. PDAF perform actual data assimilation based on its internal counter for the time steps. In `pyPDAF.init` function, we specify the initial time step. Now, a user-supplied function informs PDAF the data assimilation is performed after `nsteps` of forecast in [`py__next_observation_pdaf`](https://yumengch.github.io/pyPDAF/user_desc/py__next_observation_pdaf.html). The internal time step counter is incremented by one whenever one calls the assimilation functions.

To implement these user-supplied functions, we define a `PdafDistributor` class.

In [ ]:
class PdafDistributor:
    """Distribute the ensemble members to the model tasks
    """
    def __init__(self, nx, ny):
        # define the model field based on the ensemble
        self.nx, self.ny = nx, ny
        self.field = np.zeros((ny, nx))

PDAF only distribute one ensemble member of state vector to the model from `init_ens_pdaf`.  The next ensemble member will be distributed by PDAF once it gets the state vector for the DA step. For example, if we need to perform DA at time step $k$ with $N_e$ ensemble members:
```
for i in range(Ne):
    - PDAF distribute the i-th ensemble member to the model at time step 0.
    - model performs forecast to k-th time step
    - PDAF collects the i-th ensemble member at k-th step
    - Users need to reset the model time step to 0.
PDAF perform DA., after all ensemble members are collected at time step k.
```

In [ ]:
class PdafDistributor(PdafDistributor):
    def distribute_state(self, dim_p, state_p):
        """PDAF will distribute state vector (state_p) to model field
        """
        self.field = state_p[:].reshape((self.ny, self.nx))
        return state_p

In this simple model example, there are no need for actual processing of the ensemble. Therefore, we only show screen output of root mean squared error based on sampled variance.

In [ ]:
class PdafDistributor(PdafDistributor):
    def initial_process(self, step, dim_p, dim_ens, dim_ens_l, dim_obs_p, state_p, uinv, ens_p, flag):
        """initial processing of the ensemble before it is distributed to model fields
        """
        print (f'RMS error according to sampled variance: {np.sqrt(np.mean(np.var(ens_p, axis=1)))}')
        return state_p, uinv, ens_p

When obtaining the initial ensemble, users also need to provide information about when do we do the next analysis based on the arrival of the new observations. In our case, we have observations for each time step, but we'd like to assimilate it every other time steps.

In [ ]:
class PdafDistributor(PdafDistributor):
    def next_observation(self, stepnow, nsteps, doexit, time):
        # next observation will arrive at `nsteps' step
        nsteps = 2
        # doexit = 0 means that PDAF will continue to distribute state
        # to model for further integrations
        doexit = 0
        # model time is not used here as we only use steps to define the time
        return nsteps, doexit, time

Here, we call `pyPDAF.init_foreacst` function where it executes our user-supplied functions. Here, the model obtains the first ensemble member and perform some initial preprocessing of the full ensemble given in `py_init_ens_pdaf`.

In [ ]:
status = 0
distributor = PdafDistributor(nx, ny)
# loop over all dimensions
with pipes() as (out, err):
    status = pyPDAF.init_forecast(distributor.next_observation,
                                  distributor.distribute_state,
                                  distributor.initial_process,
                                  status)
    print (out.read())
# put model variable in distributor back to model
field = distributor.field

### Sequential data assimilation system

Data assimilation combines the model forecast and the observations. Hence, to perform data assimilation, at each analysis step, PDAF must collect the new forecast from the model and read observations. The assimilation also needs a function that connects the forecast state to observations which usually is denoted as observation operator. To ensure the flexibility of the framework, these information depends on the user-supplied functions. This is because each observation type and model are different so one should expect the users can handle these information.

In the serial system, the cycling can be divided into following steps:
1. model forecast using `step` function
2. put model forecast into PDAF state vector/ensemble array to perform DA using [`pyPDAF.assimilate`](https://yumengch.github.io/pyPDAF/_autosummary/pyPDAF.assimilate.html#pyPDAF.assimilate).

In above steps, the model forecast function is implemented in [`step`](#Model) function.

#### User-supplied functions for `pyPDAF.assimilate`
In the assimilation step, the two primary purposes is defined by `PdafCollector` and `Obs` classes. The `PdafCollector` class obtains the model forecast and the `Obs` will handle observations using the `Observation Module Infrastructure` in PDAF, a scheme to ease the difficulty in handling observations.

In [ ]:
class PdafCollector:
    def __init__(self, nx, ny, field):
        self.nx = nx
        self.ny = ny
        # define the model field based on the ensemble
        self.field = field

class Obs:
    def __init__(self, i_obs):
        # i_obs-th observations in the system starting from 1
        self.i_obs = i_obs

##### Collecting forecast
Before going into the details of observation handling, we first
1. get functions that collects the model forecast [(`collect_state_pdaf`)](https://yumengch.github.io/pyPDAF/user_desc/py__collect_state_pdaf.html). Similar to `distribute_state_pdaf`, the state vector is collected for each ensemble member from model field.
2. preprocess the ensemble forecast before the data assimilation. In the pre-processing step, we calculate the forecast error and save the forecast ensemble. This is the last step before assimilation so it could help us understand the raw forecast data. In the pre-process, the `step` argument is negative
3. post-process the analysis ensemble after the assimilation. In this case, the `step` argument is positive.

In [ ]:
class PdafCollector(PdafCollector):
    def collect_state(self, dim_p, state_p):
        """PDAF will collect state vector (state_p) from model field
        """
        state_p[:] = self.field.ravel()
        return state_p

    def preprocess(self, step, dim_ens, ens_p):
        """preprocessing of the ensemble before it is used by DA algorithms
        """
        print (f'Forecast RMS error according to sampled variance: {np.sqrt(np.mean(np.var(ens_p, axis=1)))}')
        for i in range(dim_ens):
            np.savetxt(os.path.join('outputs', f'ens_{i+1}_step{-step}_for.txt') , ens_p[:, i].reshape((self.ny, self.nx)) )

    def postprocess(self, step, dim_ens, ens_p):
        """post-processing of the ensemble before it is distributed to model fields
        """
        print (f'Analysis RMS error according to sampled variance: {np.sqrt(np.mean(np.var(ens_p, axis=1)))}')
        for i in range(dim_ens):
            np.savetxt(os.path.join('outputs', f'ens_{i+1}_step{step}_ana.txt' ), ens_p[:, i].reshape((self.ny, self.nx)) )

    def prepostprocess(self, step, dim_p, dim_ens, dim_ens_p, dim_obs_p, state_p, uinv, ens_p, flag):
        if step < 0:
          self.preprocess(step, dim_ens, ens_p)
        else:
          self.postprocess(step, dim_ens, ens_p)
        return state_p, uinv, ens_p


##### Handling observations
Another essential ingredient of data assimilation is observation. Here, user-supplied functions give all information about the observations to PDAF. We use the OMI scheme in PDAF to handle observations. Without any localisations, only two user-supplied functions are required with the OMI scheme.

Before we use the OMI scheme, we need to provide the number of observation types by [`pyPDAF.PDAFomi.init`](https://yumengch.github.io/pyPDAF/_autosummary/pyPDAF.PDAFomi.init.html#pyPDAF.PDAFomi.init). This function

Here, we use only one type of observations.

In [ ]:
pyPDAF.PDAFomi.init(1)

In this very simple example, the OMI observation functions may look a bit verbose, but it can be useful for more complex systems.

The OMI has four mandatory properties:
- [`doassim`](https://yumengch.github.io/pyPDAF/_autosummary/pyPDAF.PDAFomi.set_doassim.html#pyPDAF.PDAFomi.set_doassim): whether this observation is assimilated. If `doassim = 1`, this observation will be assimilated. If `doassim = 0`, it will not be assimilated.
- [`disttype`](https://yumengch.github.io/pyPDAF/_autosummary/pyPDAF.PDAFomi.set_disttype.html#pyPDAF.PDAFomi.set_disttype): In localisation, how do we calculate the distance between grid points? e.g., Cartesian, geographic, or great circle distance on a sphere. We do not use localisation in this example, but we still have to provide this option.
- [`ncoord`](https://yumengch.github.io/pyPDAF/_autosummary/pyPDAF.PDAFomi.set_ncoord.html#pyPDAF.PDAFomi.set_ncoord): Number of coordinates used for computation in localisation. In our example, as we have a 2D domain, the number should be 2.
- [`id_obs_p`](https://yumengch.github.io/pyPDAF/_autosummary/pyPDAF.PDAFomi.set_id_obs_p.html#pyPDAF.PDAFomi.set_id_obs_p): Indices of observed field in state vector. This is a 2D array that should have the same length as the observector vector for each dimension. If the observations do not need interpolation (e.g., observations are co-located with model grid points), the first dimension is 1. In this case, if the i-th observation is at the j-th element of the state vector, the i-th element of `id_obs_p` is `j`. If interpolation is needed, each dimension is the adjacent model grid points.

In Fortran, these properties can be given to derived type `obs_f`. In the pyPDAF, setter functions are provided. In the [`pyPDAF.PDAFomi.gather_obs`](https://yumengch.github.io/pyPDAF/_autosummary/pyPDAF.PDAFomi.gather_obs.html#pyPDAF.PDAFomi.gather_obs) function, PDAFomi collects
- the observation vector
- error variance
- the spatial coordinate of the observations

This function also returns the dimension of the observation for given observation type.

In [ ]:
class Obs(Obs):
    def init_dim(self, step, dim_obs):
        # We always assimilate the observation
        pyPDAF.PDAFomi.set_doassim(self.i_obs, 1)
        # Type of distance computation to use for localization
        # It is mandatory for OMI even if we don't use localisation
        pyPDAF.PDAFomi.set_disttype(self.i_obs, 0)
        # Number of coordinates use for distance computation
        pyPDAF.PDAFomi.set_ncoord(self.i_obs, 2)

        # read observations
        obs = np.loadtxt(os.path.join('inputs_online', f'obs_step{step}.txt'))
        # get the dimension of the model grid
        ny, nx = obs.shape
        # flatten the observations
        obs = obs.ravel()
        # a mask for observed gridpoints
        condition = np.logical_not(np.isclose(obs, -999))

        # observation vector
        y = obs[condition]

        # The relationship between observation and state vector
        # we only have 28 osbervations and each observation corresponds to
        # the grid point of one element in the state vector
        # id_obs_p gives the indices of observed field in state vector
        # the id starts from 1
        # ensure 2D arryas are in Fortran order
        id_obs_p = np.zeros((1, len(y)), dtype=np.intc, order='F')
        id_obs_p[0] = np.arange(1, len(obs) + 1, dtype=np.intc)[condition]
        pyPDAF.PDAFomi.set_id_obs_p(self.i_obs, 1, len(y), id_obs_p)

        # inverse of observation variance
        ivar_obs_p = 1./0.5/0.5*np.ones_like(y)

        # coordinate of each observations
        ocoord_p = np.zeros((2, len(y)), order='F')
        ocoord_p[0] = np.tile(np.arange(nx), ny)[condition]
        ocoord_p[1] = np.repeat(np.arange(ny), nx)[condition]

        # not being used here, only used for localisation
        local_range = 0.
        dim_obs = pyPDAF.PDAFomi.gather_obs(self.i_obs, len(y), y,
                                     ivar_obs_p, ocoord_p, 2, local_range)
        return dim_obs

The other user-supplied function in this example will be the observation operator. In this simple example, the state vector in the observation space can be conveniently obtained by the OMI function [`pyPDAF.PDAFomi.obs_op_gridpoint`](https://yumengch.github.io/pyPDAF/_autosummary/pyPDAF.PDAFomi.obs_op_gridpoint.html#pyPDAF.PDAFomi.obs_op_gridpoint) using the information provided in the `init_dim_obs` function. This function provides an observation operator where observations are located on the model grid points. When observations are not located on model grid points, PDAFomi also provides functions using linear interpolations on various grid. In this case, one may need to refer to [PDAF documentation](https://pdaf.awi.de/trac/wiki/OMI_observation_operators#Initializinginterpolationcoefficients) and [`id_obs_p` doc](https://yumengch.github.io/pyPDAF/_autosummary/pyPDAF.PDAFomi.set_id_obs_p.html#pyPDAF.PDAFomi.set_id_obs_p) for a better explanation.

In [ ]:
class Obs(Obs):
    def op(self, step, dim_p, dim_obs_p, state_p, ostate):
        """observation operator
        """
        return pyPDAF.PDAFomi.obs_op_gridpoint(self.i_obs, state_p, ostate)

##### Forward loop

Now, we can write code for the sequential DA system using [`pyPDAF.assimilate`](https://yumengch.github.io/pyPDAF/_autosummary/pyPDAF.assimilate.html#pyPDAF.assimilate). This function is universal for all available filters in PDAF that use diagnoal observation error covariance matrix. Hence, it asks for a few user-supplied functions that are meant to be used for domain localisation. In this tutorial, we can simply provide dummy functions here.

In [ ]:
def init_n_domains_p_pdaf(): pass
def init_dim_l_pdaf(): pass
def init_dim_obs_l_pdaf(): pass

In [ ]:
os.makedirs('outputs', exist_ok=True)

In [ ]:
# create directory to
current_step = 0
steps_for = 2
# get model field distributed by PDAF from `init_forecast`
field = distributor.field
obs = Obs(1)
# full DA system integration loop
while current_step < nsteps:

    for i in range(dim_ens):
        # model integration
        if i == dim_ens - 1: print('start DA')
        for _ in range(steps_for):
            field = step(field)

            # PDAF does assimilation
            with pipes() as (out, err):
                collector = PdafCollector(nx, ny, field)
                status = pyPDAF.assimilate(collector.collect_state,
                                           distributor.distribute_state,
                                           obs.init_dim, obs.op,
                                           init_n_domains_p_pdaf,
                                           init_dim_l_pdaf, init_dim_obs_l_pdaf,
                                           collector.prepostprocess,
                                           distributor.next_observation, status)
            s = out.read()
            print (s)
        field = distributor.field
    current_step += steps_for

The above loop is for illustration purpose of the serial model forecast, collection and distribution of state vector of an ensemble. One can also perform ensemble DA with a single process with assisted function [`pyPDAF.get_fcst_info`](https://yumengch.github.io/pyPDAF/_autosummary/pyPDAF.get_fcst_info.html#pyPDAF.get_fcst_info), e.g.,
```Python
doexit = 0
while True:
    if doexit == 1: break
    for _ in range(steps_for):
        field = step(field)

        # PDAF does assimilation
        with pipes() as (out, err):
            status = pyPDAF.assimilate(collector.collect_state,
                                        distributor.distribute_state,
                                        obs.init_dim, obs.op,
                                        init_n_domains_p_pdaf,
                                        init_dim_l_pdaf, init_dim_obs_l_pdaf,
                                        collector.prepostprocess,
                                        distributor.next_observation, status)
        s = out.read()
        print (s)

    steps_for, time, doexit = pyPDAF.get_fcst_info(steps_for, time, doexit)
```

## Finalise PDAF

At the end of the data assimilation program, one should finalise the PDAF system using [`pyPDAF.PDAF.deallocate`](https://yumengch.github.io/pyPDAF/_autosummary/pyPDAF.PDAF.deallocate.html#pyPDAF.PDAF.deallocate). One can also obtain screen output of the computational time and memory use information using [pyPDAF.PDAF.print_info](https://yumengch.github.io/pyPDAF/_autosummary/pyPDAF.PDAF.print_info.html#pyPDAF.PDAF.print_info)

In [ ]:
with pipes() as (out, err):
    pyPDAF.deallocate()
    pyPDAF.PDAF.print_info(1)
print (out.read())

### Does analysis look better than forecast?

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
from IPython.display import HTML
dim_ens=9
ny, nx = 18, 36
# define diagnotics and model fields
spread = {'fcst': np.zeros(9), 'ana': np.zeros(9)}
RMSE = {'fcst': np.zeros(9), 'ana': np.zeros(9)}
field = {'truth': np.zeros((ny, nx)),
         'fcst': np.zeros((dim_ens, ny, nx)),
         'ana': np.zeros((dim_ens, ny, nx))
         }
for key in spread:
    spread[key][:] = np.nan
    RMSE[key][:] = np.nan
# time
time = np.arange(2, 20, 2)

# get figure
fig = plt.figure('err')
w, h = fig.get_size_inches()
fig.set_size_inches(2*w, 2*h)
# define the time series plot
ax = fig.add_subplot(212)
ax.set_title('Time series of the ensemble spread and RMSE')
ax.set_ylim([0., 1.2])
ax.set_xlim([time[0] - 1, time[-1] + 1])
lines = []
for key, c in zip(spread, ['k', 'r']):
    line, = ax.plot(time, spread[key], color=c, linestyle='dashed',label=f'{key} spread')
    lines.append(line)
    line, = ax.plot(time, RMSE[key], color=c, linestyle='solid',label=f'{key} RMSE')
    lines.append(line)
ax.legend()
# define pcolormesh plots
ax = {'fcst': fig.add_subplot(221), 'ana': fig.add_subplot(222),}
pc = dict()
for key in ax:
    pc[key] = ax[key].pcolormesh(field[key].mean(0) - field['truth'],
                                 cmap='coolwarm', vmin=-.06, vmax=.06)
    fig.colorbar(pc[key], ax=ax[key])

def draw_error(i):
    """Draw error at each analysis time step
    """
    field['truth'] = np.loadtxt(os.path.join('inputs_online', f'true_step{i}.txt'))
    for j in range(1, dim_ens + 1):
        field['fcst'][j-1] = np.loadtxt(os.path.join('outputs', f'ens_{j}_step{i}_for.txt'))
        field['ana'][j-1]  = np.loadtxt(os.path.join('outputs', f'ens_{j}_step{i}_ana.txt'))

    for j, key in enumerate(spread):
        spread[key][i//2 - 1] = field[key].std(0).mean()
        RMSE[key][i//2 - 1] = np.sqrt(np.mean((field[key].mean(0) - field['truth'])**2))
        lines[2*j].set_ydata(spread[key])
        lines[2*j + 1].set_ydata(RMSE[key])

    for key in ax:
        ax[key].set_title(f'{key} error ({np.round(RMSE[key][i//2 - 1], 3)})')
        pc[key].set_array(field[key].mean(0) - field['truth'])

    return pc['fcst'], pc['ana'], *lines

# make an animation
anim = animation.FuncAnimation(fig, draw_error, frames=time, interval=1000, blit=True)
plt.close(fig)
HTML(anim.to_html5_video())